<a href="https://colab.research.google.com/github/woranov/amlta-project/blob/main/notebooks/tapas_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    import amlta  # noqa: F401
except ImportError:
    %pip install --no-dependencies -U git+https://github.com/woranov/amlta-project.git

  Cloning https://github.com/woranov/amlta-project.git to /tmp/pip-req-build-3x2kwxmv
  Running command git clone --filter=blob:none --quiet https://github.com/woranov/amlta-project.git /tmp/pip-req-build-3x2kwxmv
  Resolved https://github.com/woranov/amlta-project.git to commit 6d73ee545d3c220a8a0ec0eb87537a2a863e242d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for amlta: filename=amlta-0.1.0-py3-none-any.whl size=29989 sha256=d6c12c5a7a95c553e16a8588a231649786117080bddfbb1cc9b283ec199ad27a
  Stored in directory: /tmp/pip-ephem-wheel-cache-bn3j6vic/wheels/3d/ca/aa/08dfcac1c046f6e9611b7206be857f9d9b52d93473f14ef64e
Successfully built amlta


In [2]:
from pathlib import Path

try:
    from google.colab import drive  # pyright: ignore[reportMissingImports]

    IN_COLAB = True
except ImportError:
    IN_COLAB = False

from amlta.config import config

In [3]:
if IN_COLAB:
    mount_point = Path("/content/drive")
    drive_path = mount_point / "MyDrive"

    # edit
    data_dir = drive_path / "uni" / "ws2425" / "amlta" / "project" / "data"

    config.update(data_dir=data_dir)

    if not mount_point.exists():
        drive.mount(str(mount_point))

Mounted at /content/drive


In [ ]:
import functools
import importlib
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import torch
from torch.optim import AdamW
from tqdm.auto import tqdm
from transformers import TapasConfig, TapasForQuestionAnswering, TapasTokenizer

from amlta.data_processing.tapas_flows import transform_flows_for_tapas
from amlta.probas import flows, processes
from amlta.question_generation.process import (
    build_training_batched_dfs_all,
    load_batches,
)

processes = importlib.reload(processes)

In [ ]:
tapas_config = TapasConfig.from_pretrained(
    "google/tapas-base-finetuned-wikisql-supervised"
)
tapas_model = TapasForQuestionAnswering.from_pretrained(
    "google/tapas-base", config=tapas_config
)
tapas_tokenizer = TapasTokenizer.from_pretrained(
    "google/tapas-base-finetuned-wikisql-supervised"
)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
tapas_model.to(device)

Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['aggregation_classifier.bias', 'aggregation_classifier.weight', 'column_output_bias', 'column_output_weights', 'output_bias', 'output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TapasForQuestionAnswering(
  (tapas): TapasModel(
    (embeddings): TapasEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (token_type_embeddings_0): Embedding(3, 768)
      (token_type_embeddings_1): Embedding(256, 768)
      (token_type_embeddings_2): Embedding(256, 768)
      (token_type_embeddings_3): Embedding(2, 768)
      (token_type_embeddings_4): Embedding(256, 768)
      (token_type_embeddings_5): Embedding(256, 768)
      (token_type_embeddings_6): Embedding(10, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): TapasEncoder(
      (layer): ModuleList(
        (0-11): 12 x TapasLayer(
          (attention): TapasAttention(
            (self): TapasSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias

In [ ]:
results = load_batches()

In [ ]:
examples_batched_df = build_training_batched_dfs_all()
examples_batched_df

,batch,question_id,question_template,question_basic,question_general,question_specific,process_uuid,flows_start,flows_stop,aggregation,coordinates,answers
0,batch_gpt-4o-mini_0_200,0,What are the total emissions to air from <the ...,What are the total emissions to air from energ...,What are the total emissions to air from solar...,What are the total emissions to air from elect...,404ee608-dcb1-4c7e-9bda-b5eafe763547,0,15,NONE,[],[]
1,batch_gpt-4o-mini_0_200,0,What are the total emissions to air from <the ...,What are the total emissions to air from energ...,What are the total emissions to air from solar...,What are the total emissions to air from elect...,404ee608-dcb1-4c7e-9bda-b5eafe763547,15,30,NONE,"[(7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12...","[3.88, 4.3, 0.0, 0.0, 0.0, 0.0, 0.008, 0.0]"
2,batch_gpt-4o-mini_0_200,0,What are the total emissions to air from <the ...,What are the total emissions to air from energ...,What are the total emissions to air from solar...,What are the total emissions to air from elect...,404ee608-dcb1-4c7e-9bda-b5eafe763547,30,45,NONE,"[(0, 1), (1, 1), (2, 1), (4, 1), (5, 1), (6, 1...","[0.0, 0.0, 0.001, 0.002, 0.0, 0.0, 15.1, 0.52,..."
3,batch_gpt-4o-mini_0_200,0,What are the total emissions to air from <the ...,What are the total emissions to air from energ...,What are the total emissions to air from solar...,What are the total emissions to air from elect...,404ee608-dcb1-4c7e-9bda-b5eafe763547,45,60,NONE,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,batch_gpt-4o-mini_0_200,0,What are the total emissions to air from <the ...,What are the total emissions to air from energ...,What are the total emissions to air from solar...,What are the total emissions to air from elect...,404ee608-dcb1-4c7e-9bda-b5eafe763547,60,70,NONE,"[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1...","[0.0, 0.0, 0.0, 7598.0, 23.9, 8.5, 0.0, 0.0, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3775,batch_gpt-4o-mini_0_200,2,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,71f0bce6-9190-4137-86f6-67e9473fb10d,15,30,COUNT,"[(8, 1), (10, 1)]","[229.0, 9.75]"
3776,batch_gpt-4o-mini_0_200,2,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,71f0bce6-9190-4137-86f6-67e9473fb10d,30,45,COUNT,[],[]
3777,batch_gpt-4o-mini_0_200,2,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,71f0bce6-9190-4137-86f6-67e9473fb10d,45,60,COUNT,[],[]
3778,batch_gpt-4o-mini_0_200,2,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,What is the total amount of waste produced by ...,71f0bce6-9190-4137-86f6-67e9473fb10d,60,75,COUNT,[],[]


In [ ]:
aggregation2id = {agg: id for id, agg in tapas_config.aggregation_labels.items()}

In [ ]:
@functools.lru_cache(maxsize=512)
def get_flows_df(uuid: str) -> pd.DataFrame:
    process = processes.ProcessData.from_uuid(uuid)
    return flows.extract_process_flows(process)


class TableDataset(torch.utils.data.Dataset):
    def __init__(
        self, df: pd.DataFrame, tokenizer: TapasTokenizer, table_chunk_size: int = 20
    ):
        super().__init__()
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if isinstance(idx, slice):
            indices = range(*idx.indices(len(self)))
            return [self[i] for i in indices]

        item = self.df.iloc[idx]

        # question_specific = item["question_specific"]
        question_general = item["question_general"]
        question_process_agnostic = (
            item["question_template"].replace("<", "").replace(">", "")
        )

        flows_df = (
            get_flows_df(item["process_uuid"])
            .iloc[item["flows_start"] : item["flows_stop"]]
            .reset_index(drop=True)
        )
        table = transform_flows_for_tapas(flows_df).astype(str)

        # display(flows_df)
        # display(table)

        encoding = self.tokenizer(
            table=table,
            queries=question_process_agnostic,
            answer_coordinates=item["coordinates"],
            answer_text=item["answers"],
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        # remove the batch dimension which the tokenizer adds by default
        encoding = {key: val.squeeze(0) for key, val in encoding.items()}
        encoding["aggregation_labels"] = aggregation2id[item["aggregation"]]

        return encoding

In [ ]:
dataset_train = TableDataset(
    df=examples_batched_df.iloc[: int(len(examples_batched_df) * 0.8)],
    tokenizer=tapas_tokenizer,
)

dataset_test = TableDataset(
    df=examples_batched_df.iloc[int(len(examples_batched_df) * 0.8) :],
    tokenizer=tapas_tokenizer,
)

In [ ]:
dataloader_train = torch.utils.data.DataLoader(
    dataset_train, batch_size=8, shuffle=True
)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=8, shuffle=False)

In [13]:
print(tapas_tokenizer.decode(dataset_train[1]["input_ids"]))

[CLS] what are the total emissions to air from the process? [SEP] direction amount type class name property unit input 7069. 77 elementary flow resources / resources from ground / non - renewable energy resources from ground crude oil net calorific value mj input 955. 474 elementary flow resources / resources from ground / non - renewable energy resources from ground natural gas net calorific value mj input 13200. 0 elementary flow resources / resources from ground / non - renewable energy resources from ground brown coal net calorific value mj input 608. 569 elementary flow resources / resources from biosphere / renewable energy resources from biosphere biomass net calorific value mj input 0. 008 elementary flow none nuclear net calorific value mj input - 0. 0 elementary flow emissions / emissions to air / emissions to air, unspecified waste heat net calorific value mj output 1000000. 0 product flow systems / other systems electricity net calorific value mj output 3. 88 elementary flo

In [95]:
optimizer = AdamW(tapas_model.parameters(), lr=5e-5)

tapas_model.train()
for epoch in range(4):  # loop over the dataset multiple times
    for batch in tqdm(dataloader_train):
        # get the inputs;
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch["labels"].to(device)
        numeric_values = batch["numeric_values"].to(device)
        numeric_values_scale = batch["numeric_values_scale"].to(device)
        aggregation_labels = batch["aggregation_labels"].to(device)
        # float_answer = batch["float_answer"]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = tapas_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels,
            numeric_values=numeric_values,
            numeric_values_scale=numeric_values_scale,
            aggregation_labels=aggregation_labels,
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch} loss: {loss.item()}")

  0%|          | 0/378 [00:00<?, ?it/s]

Epoch 0 loss: 0.19582802057266235


  0%|          | 0/378 [00:00<?, ?it/s]

Epoch 1 loss: 0.09167389571666718


  0%|          | 0/378 [00:00<?, ?it/s]

Epoch 2 loss: 1.3120675086975098


  0%|          | 0/378 [00:00<?, ?it/s]

Epoch 3 loss: 0.1167212426662445


In [ ]:
tapas_model.eval()

ex = examples_batched_df.iloc[int(len(examples_batched_df) * 0.8) :].sample(1).iloc[0]
question_general = ex["question_general"]
question_process_agnostic = ex["question_template"].replace("<", "").replace(">", "")

flows_df = (
    get_flows_df(ex["process_uuid"])
    .iloc[ex["flows_start"] : ex["flows_stop"]]
    .reset_index(drop=True)
)
table = transform_flows_for_tapas(flows_df).astype(str)

queries = question_process_agnostic

display(table)
print(queries)

inputs = tapas_tokenizer(
    table=table, queries=queries, padding="max_length", return_tensors="pt"
)
outputs = tapas_model(**inputs.to(device))
predicted_answer_coordinates, predicted_aggregation_indices = (
    tapas_tokenizer.convert_logits_to_predictions(
        inputs.to("cpu"),
        outputs.logits.detach().to("cpu"),
        outputs.logits_aggregation.detach().to("cpu"),
        cell_classification_threshold=0.25,
    )
)

# let's print out the results:
id2aggregation = {0: "NONE", 1: "SUM", 2: "AVERAGE", 3: "COUNT"}
aggregation_predictions_string = [
    id2aggregation[x] for x in predicted_aggregation_indices
]

print(aggregation_predictions_string)
print(predicted_answer_coordinates)

print(ex["coordinates"])

,Direction,Amount,Type,Class,Name,Property,Unit
0,Output,0.0,Elementary flow,Emissions / Emissions to air / Emissions to ai...,nickel,Mass,kg
1,Output,0.0,Elementary flow,Emissions / Emissions to air / Emissions to ai...,ammonia,Mass,kg
2,Output,-0.0,Elementary flow,Emissions / Emissions to air / Emissions to ai...,nitrous oxide,Mass,kg
3,Output,0.0,Elementary flow,Emissions / Emissions to water / Emissions to ...,"nitrogen, total (excluding N2)",Mass,kg
4,Output,0.0,Elementary flow,None,waste-nuclear fuel,Mass,kg
5,Output,0.0,Elementary flow,Emissions / Emissions to air / Emissions to ai...,mercury,Mass,kg
6,Output,0.0,Elementary flow,Emissions / Emissions to water / Emissions to ...,mercury,Mass,kg
7,Output,0.0,Elementary flow,Emissions / Emissions to air / Emissions to ai...,HFC-4310mee,Mass,kg
8,Output,0.0,Elementary flow,Emissions / Emissions to air / Emissions to ai...,HFC-32,Mass,kg
9,Output,0.0,Elementary flow,Emissions / Emissions to air / Emissions to ai...,HFC-245ca,Mass,kg


What is the total count of emissions outputs from the process?
['COUNT']
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)]]
[(0, 1), (1, 1), (2, 1), (3, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)]


In [135]:
ex

,3305
batch,batch_gpt-4o-mini_0_200
question_id,2
question_template,What are the amounts of primary energy from re...
question_basic,What are the amounts of primary energy from re...
question_general,What are the amounts of primary energy from re...
question_specific,What are the amounts of primary energy from re...
process_uuid,aab4825f-87a9-4a58-8701-649e83c7a4c8
flows_start,75
flows_stop,79
aggregation,NONE


In [ ]:
params = next(
    res
    for res in results
    if res["process_uuid"] == ex["process_uuid"]
    and res["question_id"] == ex["question_id"]
)

In [137]:
params

{'batch': 'batch_gpt-4o-mini_0_200',
 'process_uuid': 'aab4825f-87a9-4a58-8701-649e83c7a4c8',
 'question_id': 2,
 'basic_query': 'chemical production processes in Germany',
 'general_query': 'oxygen production process',
 'specific_query': 'liquid oxygen production in Germany 2000',
 'flow_query_params': {'query_type': 'type',
  'direction': 'both',
  'aggregation': 'list',
  'flow_type': 'Elementary flow'},
 'question': 'What are the amounts of primary energy from renewable sources used in <the process>?',
 'question_replaced_basic': 'What are the amounts of primary energy from renewable sources used in chemical production processes in Germany?',
 'question_replaced_general': 'What are the amounts of primary energy from renewable sources used in oxygen production process?',
 'question_replaced_specific': 'What are the amounts of primary energy from renewable sources used in liquid oxygen production in Germany 2000?'}